<a href="https://colab.research.google.com/github/PEREZRTbc/MLAI-Coding-Project/blob/main/CSCI_3349_S25_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: nltk and spaCy

In this lab, you will be learning a bit about how to use the Python libraries <code>nltk</code> and ``spaCy`` to perform text normalization and to explore and analyze texts.

The first few parts of this notebook will help you understand how to use a Colab notebook. Note that if you ever get weird behavior in a notebook. just go up to the Runtime menu and Restart, then run each code cell up to where you started having the problem.

**After** you have completed all code and questions in this notebook, you must do two things:

1. Share the notebook with the me and the TAs: prudhome@bc.edu, mccullkg@bc.edu, and pawsat@bc.edu.
2. Download this file to your computer with `File-> Download .ipynb`, move it to your repo for this lab, then add, push, and commit your version of this Colab notebook.

The deadline is Monday, January 27, at 11:59pm EST.

## 1. Getting started

1. Make your own copy of this notebook by going to `File->Save a copy in Drive`. This will create your own copy of this notebook that you can edit and run on Colab.

2. Click on the title of the notebook, up above, and change it to `YourLastName_YourFirstName_Lab2.ipynb`.

3. The cell below is a code cell. Click in the cell, then type <code>print("Hello, World!")</code> in that cell under the comment. Then, hit the run button just to the left of the comment that looks like a white triangle pointing to the right inside a black circle. The keyboard shortcut is <code>shift-return</code>, holding both keys down at the same time.

In [ ]:
# enter your Hello World code here



Underneath your command you should now see the output <code>Hello, World!</code>.

Great! Now you have run your first command in this Colab Notebook. You can always go back and edit the stuff you've written in any code cell. Just remember to re-run it if you change anything.

*Note: Many notebook beginners forget that if you change the value of some variable in a block of code, that variable now has that new value everywhere -- even in earlier blocks of code. If you are having trouble, it often helps to go back and re-run the block of code where you originally set the value of that variable.*

Now let's start using nltk. Type <code>import nltk</code> in the command cell below and then run the cell by clicking the run button to import the nltk library

In [ ]:
# enter your import nltk command here and hit Run




You might not have all the packages you need by default in nltk. Just in case, you should download the most popular ones.

Note that it might take a little while to do the download. Wait until you see the little green checkmark next to the `[ ]` before you go on to the next cell.

In [ ]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('popular')
nltk.download('punkt_tab')


Just to make sure your nltk is working, use it to calculate the minimum edit distance between two words. The function is <code>nltk.edit_distance</code> and the arguments are the two strings you want to compare.

In [ ]:
# enter your call to edit distance here and hit Run



We're going to be using regular expressions, so we'll import tbe `re` library, too.

In [ ]:
# enter your import re command here



Now let's download a file. We're going to look at Great Expectations, by Charles Dickens. Click on the cell below and hit the Run button to issue the command to download the plain text version of the book from Project Gutenberg.

This is one of the very cool things about Colab notebooks: you can issue (some) unix commands by preceding the command with an exclamation point. Commands involving the file system (cd, mv, etc.) are trickier in Colab. We'll learn more about how Colab interacts with the Google Drive file system in a future lab.

In [ ]:
# You can put a ! before a unix command and see if it works!
# curl is one that usually works the way you are expecting

! curl -o greatexpectations.txt https://www.gutenberg.org/files/1400/1400-0.txt

Now you have a text to work with. Again, I am punting on teaching you about the Colab vs. Google Drive file systems, but you can see that the file `greatexpectations.txt` got saved to your current directory using the unix command `ls`. You should also examine the first 100 lines and the last 100 lines of the file using `head` and `tail`. (Don't forget the exclamation point before!)

In [ ]:
# below issue unix commands preceded by ! to
# (1) see what's in your current directory
# (2) examine the first 100 lines of greatexpectations.txt
# (3) examine the last 100 lines of greatexpectations.txt






## 2. Loading in the text

You'll notice that plain text Gutenberg Project books are formatted to have 80 or fewer characters per line. This is fine for reading on an old-timey computer screen, but when we're processing text, we don't want a lot of manually inserted hard line breaks in the middle of our text. We're going to read in the text and replace line breaks with spaces. Run the code below.

In [ ]:
f = open("greatexpectations.txt", "r", encoding="utf-8")
thetext = f.read().rstrip()
thetext = re.sub("\n", " ", thetext)
f.close()

<code>thetext</code> is a single string containing the entire text of the book. You can see that this is true by printing out the whole thing, but that will take up lots of space. Instead just try printing a few slices like this. This prints the first 25 characters and the last 99 characters.

In [ ]:
print(thetext[0:25])
print(thetext[-99:])

Recall from when you examined the file in a text editor that there there was a bunch of text at the beginning and end of the file that was not actually a part of the text of the book. Above I showed how to use <code>re.sub</code> to remove all the line breaks. In the cell below, use <code>re.sub</code> to delete everything up to and including ``Chapter I.   `` **followed by three spaces**. Then use <code>re.sub</code> to delete everything starting from the white space that appears before ``*** END OF THE PROJECT GUTENBERG EBOOK GREAT EXPECTATIONS ***`` all the way to the end of the file.

Hint: Be very careful about spaces, case, punctuation, etc. Some regular expressions you will find very useful: <code>+ ^ $ \s .\*</code> and <code>.\*?</code> and the backslash.

Save this updated version to a variable called `alltext`.

In [ ]:
# enter your code here and run it to create the alltext variable




If you did your regular expressions right, repeating the slice printing commands above will yield the following output:

<code>My father’s family name b</code><br>
<code>the broad expanse of tranquil light they showed to me, I saw no shadow of another parting from her.</code>


## 3. Word tokenization

In Python, you can turn a "sentence" into a string of "words" by splitting on white space using the <code>split</code> function. As we've discussed in class, however, splitting on white space is not a great way to tokenize (i.e., to separate out each actual word) because you leave punctuation attached to words. This prevents you from recognizing that, for instance, "dogs" is the same word whether it's before a space or a comma. In addition, you won't be able to learn anything about the distribution of different punctuation marks since they will always be attached to something else.

Fortunately, nltk has a word tokenizer function that, when given a string, will return a list of tokens. Here's the syntax for calling it:

<code>listoftokens = nltk.word_tokenize(inputstring)</code>

Call this function on <code>alltext</code> to produce a list of tokens called <code>alltokens</code>.

In [ ]:
# call nltk.word_tokenize here and Run

alltokens = nltk.word_tokenize(alltext)


### Q1: How many tokens are there in this text? How many types are there in this text? What is the type:token ratio? Write three python commands in the line below that will calculate these three numbers. Then print out all three numbers.

In [ ]:
# line of code for token count



# line of code for type count
# (Hint: remember about sets, which you learned in CS1)




# line of code for type:token ratio



# line of code to print out all three




### Q2: What text normalization might you want to do before counting the number of types and tokens? (Hint: there are some words you might be counting as separate types because of the way they are spelled even though you'd probably think of them as the same word.) How might this normalization make your type and token counts more accurate? How might it make these counts less accurate?

### Double click here to enter your answers to Q2
  

## 4. Frequency distributions

Your answers to Q1 demonstrate that there must be some words that were used more than once. Suppose you want to know what are the most frequent words. You can do this using the <code>FreqDist()</code> class in nltk. Run the code below to create a frequency distribution for your list of tokens and to print out the 10 most frequent words and their counts.

In [ ]:
fdist = nltk.FreqDist(alltokens)
fdist.most_common(10)


It's not too surprising that the words you see in this list are the most common words. These little words that don't add a lot of content to language but appear frequently and usually serve a specific function are called <i><b>function words</i></b> or <i><b>closed class words</i></b>. These words are important, but the don't tell us much by themselves about the story.

What should we do if we want to know the most frequent words that are <i><b>content words</b></i> or <i><b>open class words</b></i> like nouns, verbs, adjectives, and adverbs -- the kinds of words that can tell us more about the story itself?

We filter out the function words using a <i><b>stop list</b></i>, which is a list of words that we can skip when we're interested in the real content of a text. nltk provides a stop list that you can use and add to. Let's get it and print it out to see what's there.

In [ ]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')
print(stoplist)


### Q3: What common and important class of tokens did you see in the most frequent token list that is missing from this list that we also might like to ignore if we are interested in content words?

### Double click here to answer Q3




Add at least three of these missing tokens to the stop list using the usual Python syntax for appending to or extended a list, and check to make sure it worked. Then make a new version of <code>alltokens</code> from which all stop words in your stoplist have been removed. Finally, create a new <code>FreqDist</code> from this stopword-free list of tokens, and print out the top 10 tokens.

Keep adding stop words (or stop tokens!) to the stoplist until you start seeing mostly real content words in the top 10.

**(Note: There are smart quotes in the text because it's UTF-8 not ascii. You can add these to the stoplist by just copying and pasting them into your list of things you're adding to the stoplist.)**

In [ ]:
stoplist = stopwords.words('english')

# enter your code for appending at least three tokens to the stop list here



# print out the stoplist to make sure your new tokens were added correctly



# make a new version of alltokens called allcontenttokens that doesn't contain items from the stop list



# create a new FreqDist from this new version of allcontenttokens



# print out the top 10 most frequent tokens in this new FreqDist



# Remember to repeat the above steps until the 10 most frequent words are content words
# rather than function words or punctuation!


### Q4: How many tokens did you have to add to the stoplist? What do you think of nltk's stoplist?

### Double click here to answer Q4




## 5. N-grams

In class, we learned about language modeling with n-grams. An n-gram is sequence of *n* words. When $n=1$, you have unigrams. When $n=2$, you have bigrams. When $n=3$, you have trigrams, and so on. You can learn a lot about a language just by looking at the frequencies of n-grams.

You can count n-grams easily yourself, but the nltk library makes counting n-grams really easy with <code>nltk.util.ngrams</code>.

The `nltk.util.ngrams()` function creates a `generator` object containing all of bigrams and trigrams. That might be helpful if we were dealing with a lot of data, but it's really not necessary here. I am converting the generator objects into lists so we can look at them and do stuff with them. You can just run the code cell below.

**Note: Below I am using ``alltokens`` here and not ``allcontenttokens``. Why? Because language models are used when we are interested in word sequences and how words fit together with each other. We don't want to throw out the function words since they tell us about the structure of the language.**


In [ ]:
mybigrams = nltk.ngrams(alltokens,2)
mytrigrams = nltk.ngrams(alltokens,3)

print(type(mybigrams))
print(type(mytrigrams))

bigramlist = list(mybigrams)
trigramlist = list(mytrigrams)

print(bigramlist[:10])
print(trigramlist[:10])


Make a `FreqDist` for the bigrams above passing in `bigramlist`. Then do the same for trigrams.

In [ ]:
# Enter your code for creating FreqDist for bigramlist and trigramlist.



# Print out the 10 most common bigrams and trigrams.





When working with bigrams in nltk, you can also build a <i>conditional</i> frequency distribution, which, for a given word, keeps track of the frequencies of any following words. Let's look.

In [ ]:
bicfreq = nltk.ConditionalFreqDist(nltk.ngrams(alltokens,2))

print(bicfreq["Mr."].most_common(10))  # prints out common words after Mr.

print(bicfreq["who"].most_common(10))  # prints out common words after who


## 6. Stemming and lemmatization

There's a common normalization task we haven't performed yet: stemming or lemmatization.

### Q5: Looking at the top 50 or 100 most frequent unigrams, how can you tell the tokens are not stemmed or lemmatized?

### Double click here to answer Q5




The command cell below shows how to use nltk's only true lemmatizer, the WordNet Lemmatizer.

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# an example
print(lemmatizer.lemmatize("dogs"))
print(lemmatizer.lemmatize("speaks", "v"))

Use this lemmatizer to lemmatize every token in the <code>allcontenttokens</code> list you created above. Then make a new frequency distribution and examine the 50 or 100 most frequent words.

**Note: I want you to use ``allcontenttokens`` here. Why? Because we are thinking about words by themselves here rather than word sequences so we can disregard function words. In addition, you can lemmatize only verbs, nouns, adjectives, and adverbs (in English, at least).**

In [ ]:
# Create a new list of tokens, all_lemmas by lemmatizing allcontenttokens




# Build a new FreqDist on all_lemmas and print out the 50
# most frequent lemmatized tokens.




It probably doesn't look much better. This is because the WordNet lemmatizer in nltk assumes by default that every word is a noun. Unless you tell the lemmatizer that something is a verb, it won't try to look it up as a verb. This is why "said" doesn't get lemmatized, and also why "was" gets lemmatized to "wa". In a future lab or problem set, we'll be exploring automatic part of speech tagging, which allows us to label every word as a noun, verb, adjective, preposition, etc. We'll also see shortly that spaCy does a much better job with this.

Note that there are several different stemmers implemented in the nltk.stem package.

## 7. Sentence tokenization

For the second part of this lab, you'll need to take this text and save it out to a file with one tokenized sentence per line. Let's start by going back to the string holding our original text, <code>alltext</code>. We can turn this into a list of strings, each of which is a sentence, using the <code>sent_tokenize()</code> function, which takes a string as an argument and returns a list of sentences.

Below, take <code>alltext</code>, break it up into sentences with <code>sent_tokenize()</code>. Then loop through the sentences in that list, and use <code>word_tokenize</code> to tokenize each sentence. Print each tokenized sentence out to a file so that you have one sentence per line.

**Note: Do not just call <code>print()</code>! This will print out an ugly list of lists. Cycle through the lists to print out strings.**

For example, this text:

<code>Open the pod bay doors, Hal! I'm sorry, Dave. I can't do that.</code>

would get printed to a file as this:

<code>Open the pod bay doors , Hal !</code><br>
<code>I'm sorry , Dave .</code><br>
<code>I can't do that .</code><br>

Please observe that there are *no quotes, no square brackets, and no commas, as you would get if you just called ``print()`` on a Python list!* TAs will be instructed to give you a 0 for this section if you print out a raw list.

Name the file you print out to <code>great.txt</code>.

In [ ]:
# use sent_tokenize() to break alltext into a list of sentences, allsent



# Open a file to write to called great.txt.




# Loop through the sentences


    # call word_tokenize() on each sentence


    # First write out <s> to the file to indicate the beginning of a sentence, then a space.


    # Then write out to the file each token one-by-one, each followed by a space.


    # Then write out </s> to indicate the end of the sentence.



# Close the file great.txt if needed.


Look at the first 10 and last 10 lines of your new `great.txt` file using unix commands in the cell below.

The second line of your file should look like this:

```
<s> So , I called myself Pip , and came to be called Pip . </s>
```

In [ ]:
# Use unix commands to look at the first 10 and last 10 lines.




## 8. Using spaCy to do a lot of this work for you

It is crucial that you understand how to do each of these steps yourself since which steps you do depends very much on what task you are working on. Things like capitalization, punctuation, function words, and sentence boundaries might be important for what you want to do, or they might not matter at all.

However, there is a different python library, spaCy, that will do a lot of this for you (and more!) automagically (and more slowly). Experiment with the code below to see the different things spaCy can do. To explore more, you can consult [the official spaCy documentation](https://spacy.io/api) or helpful websites like [this](https://realpython.com/natural-language-processing-spacy-python/).

**You might get a warning or other feedback saying you need to restart your runtime. I was able to get everything to work without restarting, but if you can't run the subsequent code, just restart and rerun (`Runtime -> Restart Session and Run All`).**

In [ ]:
import spacy

# This line below will download the English models.
# They appear to already be available on Colab, but if you run into
# problems you can uncomment this line.
# You may also need to download models when you get to the part
# below where you look at another language. You can use the code
# below as a model.
#!python -m spacy download en_core_web_sm

# This line loads a big model/pipeline that works specifically for English.
nlp = spacy.load('en_core_web_sm')

# Remember: spaCy is fancy, so it can be slow. Let's look at just the
# first 10000 characters of Great Expectations.

doc = nlp(alltext[0:10000])



The pipeline you loaded in line 9 in the above code block, which I called ``nlp``, takes as input a text. It then returns a data structure that contains a very detailed processing and analysis of that text, including sentence boundary detection, tokenization, lemmatizing, part of speech  tagging, and all kinds of other helpful things.

In [ ]:
# Here's how to get access to sentences.
for sent in doc.sents:
    print(sent)



In [ ]:
# Here's how to get tokens, along with information
# about each token such as its lemma and part of speech.
# I am just printing out the first 10!
for token in doc[:10]:
    print(token, token.lemma_, token.pos_)


In [ ]:
# spaCy has stoplists, too, and they are much more
# complete and expansive (perhaps too expansive)
# than the nltk list

english_stops = spacy.lang.en.stop_words.STOP_WORDS
print(english_stops)

To get some practice using spaCy, I'd like you to try it out some of the above commands but with a new text in a language that is *not* English!

**Step 1**: There are many trained pipelines for different languages here: https://spacy.io/models. Go to that address, and pick a language. In the code box below where you pick the language, you'll see a line that shows you how to load a pipeline for the language you have selected (e.g., ``nlp = spacy.load("es_core_news_sm")`` for Spanish).

**Step 2**: Go on the web and find a chunk of text for the language you chose. You can pick text from Gutenberg or from Google news for that langauge or from any website where you can get a good continuous chunk of 100-200 words.

**Step 3**: Process that chunk of text with the language pipeline you chose in Step 1. (You can just copy and paste the chunk into your code block.)

**Step 4**: Print out the following in a pleasing way (i.e., no raw lists or other data structures):
* The number of tokens in the text.
* The number of sentences in the text.
* All the verbs in the text, with each verb being printed only once.
* All the stopwords in the text, with each stopword being printed only once.


In [ ]:
# Write your code for Part 8 here. Do not forget to include comments!




## 9. Verifying and submitting your work

1. Make sure you've answered every <b>Q</b> question.

2. Make sure you've written code wherever required.

3. Go up to the `Runtime` menu and select `Restart Session and Run All`. This will run all of the code you've written. Make sure there are no errors.

4. Download this Colab notebook to your computer by going to `File -> Download -> Download .ipynb`.

5. Move your downloaded notebook to your lab 2 repo.

6. Add, commit, and push.

7. Share the notebook with the me and the TAs: prudhome@bc.edu, mccullkg@bc.edu, and pawsat@bc.edu.

This lab is due January 27 at 11:59pm EST.